# This file contains implementation of loan eligibility

The data is from: https://www.kaggle.com/datasets/vikasukani/loan-eligible-dataset/code

The project uses three models:
* Logistic Regression
* Support Vector Machine
* Neural Networks

In [187]:
# import important libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from ast import increment_lineno
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn import svm

# Step 1: View the data

In [180]:
train_data = pd.read_csv('loan-train.csv')

In [162]:
train_data.head()

Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001002   Male      No          0      Graduate            No   
1  LP001003   Male     Yes          1      Graduate            No   
2  LP001005   Male     Yes          0      Graduate           Yes   
3  LP001006   Male     Yes          0  Not Graduate            No   
4  LP001008   Male      No          0      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History Property_Area Loan_Status  
0             1.0         Urban           Y  
1             1.0         Rural           N  
2             1.0         Urban           Y  
3             1.0         Urban           Y  
4             1.0         Urban           Y

# Step 2: Preprocess the data

In [163]:
# Drop unwanted columns from datasets
train_data = train_data.drop('Loan_ID', axis=1)

In [164]:
# check for null values
train_data.isna().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [165]:
# fill the null values with the mean of each column
train_data.fillna(train_data.mean(), inplace=True)

train_data['Dependents'].fillna(train_data['Dependents'].mode()[0], inplace=True)

C:\Users\Ammar\AppData\Local\Temp/ipykernel_36428/874459587.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_data.fillna(train_data.mean(), inplace=True)


In [166]:
train_data.isna().sum()

Gender               13
Married               3
Dependents            0
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History        0
Property_Area         0
Loan_Status           0
dtype: int64

In [167]:
# check the values for non numerical columns
print(train_data['Gender'].unique())
print(train_data['Married'].unique())
print(train_data['Education'].unique())
print(train_data['Self_Employed'].unique())
print(train_data['Loan_Status'].unique())
print(train_data['Property_Area'].unique())
print(train_data['Dependents'].unique())

['Male' 'Female' nan]
['No' 'Yes' nan]
['Graduate' 'Not Graduate']
['No' 'Yes' nan]
['Y' 'N']
['Urban' 'Rural' 'Semiurban']
['0' '1' '2' '3+']


In [168]:
# drop rows with nan values in Married and Gender column
train_data = train_data.dropna(subset=['Married'])
train_data = train_data.dropna(subset=['Gender'])
train_data = train_data.dropna(subset=['Self_Employed'])

In [169]:
train_data.isna().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [170]:
# Convert the non-numerical values to numerical values
train_data['Gender'] = train_data['Gender'].replace({'Male': 1, 'Female': 0})
train_data['Married'] = train_data['Married'].replace({'Yes': 1, 'No': 0})
train_data['Education'] = train_data['Education'].replace({'Graduate': 1, 'Not Graduate': 0})
train_data['Self_Employed'] = train_data['Self_Employed'].replace({'Yes': 1, 'No': 0})
train_data['Loan_Status'] = train_data['Loan_Status'].replace({'Y': 1, 'N': 0})
train_data['Property_Area'] = train_data['Property_Area'].replace({'Urban': 0, 'Rural': 1, 'Semiurban': 2})
train_data['Dependents'] = train_data['Dependents'].replace({'0':0, '1':1, '2':2, '3+':3})

In [171]:
train_data.head()

Gender  Married  Dependents  Education  Self_Employed  ApplicantIncome  \
0       1        0           0          1              0             5849   
1       1        1           1          1              0             4583   
2       1        1           0          1              1             3000   
3       1        1           0          0              0             2583   
4       1        0           0          1              0             6000   

   CoapplicantIncome  LoanAmount  Loan_Amount_Term  Credit_History  \
0                0.0  146.412162             360.0             1.0   
1             1508.0  128.000000             360.0             1.0   
2                0.0   66.000000             360.0             1.0   
3             2358.0  120.000000             360.0             1.0   
4                0.0  141.000000             360.0             1.0   

   Property_Area  Loan_Status  
0              0            1  
1              1            0  
2              0            1  
3              0            1  
4              0            1

In [172]:
# split the data
X = train_data.iloc[:, :-1]
y = train_data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [174]:
# Normalize data for better training
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 3: Model Training

# 3.1 Logistic Regression

In [181]:
lr=LogisticRegression()

In [182]:
lr.fit(X_train,y_train)

LogisticRegression()

In [183]:
y_predict=lr.predict(X_test)

In [186]:
print('Accuracy:', accuracy_score(y_test,y_predict)*100)

Accuracy: 78.0701754385965


# 3.2 Support Vector Machine

In [189]:
classifier=svm.SVC(kernel="linear")
classifier.fit(X_train,y_train)

SVC(kernel='linear')

In [191]:
y_pred=classifier.predict(X_test)
from sklearn import metrics
print ("accuracy:",metrics.accuracy_score(y_test,y_pred))

accuracy: 0.7982456140350878


# 3.3 Neural Networks

In [192]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [193]:
# Train the model
history = model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=32)

Epoch 1/100
12/12 [==============================] - 0s 10ms/step - loss: 0.6859 - accuracy: 0.5789 - val_loss: 0.6429 - val_accuracy: 0.7363
Epoch 2/100
12/12 [==============================] - 0s 2ms/step - loss: 0.5916 - accuracy: 0.7950 - val_loss: 0.5772 - val_accuracy: 0.7912
Epoch 3/100
12/12 [==============================] - 0s 2ms/step - loss: 0.5263 - accuracy: 0.8199 - val_loss: 0.5360 - val_accuracy: 0.8132
Epoch 4/100
12/12 [==============================] - 0s 2ms/step - loss: 0.4810 - accuracy: 0.8172 - val_loss: 0.5167 - val_accuracy: 0.8022
Epoch 5/100
12/12 [==============================] - 0s 2ms/step - loss: 0.4512 - accuracy: 0.8172 - val_loss: 0.5066 - val_accuracy: 0.8022
Epoch 6/100
12/12 [==============================] - 0s 2ms/step - loss: 0.4380 - accuracy: 0.8172 - val_loss: 0.4999 - val_accuracy: 0.7912
Epoch 7/100
12/12 [==============================] - 0s 2ms/step - loss: 0.4277 - accuracy: 0.8172 - val_loss: 0.5020 - val_accuracy: 0.7912
Epoch 8/100


Epoch 59/100
12/12 [==============================] - 0s 2ms/step - loss: 0.2614 - accuracy: 0.8947 - val_loss: 0.6384 - val_accuracy: 0.7802
Epoch 60/100
12/12 [==============================] - 0s 2ms/step - loss: 0.2620 - accuracy: 0.8892 - val_loss: 0.6404 - val_accuracy: 0.7802
Epoch 61/100
12/12 [==============================] - 0s 3ms/step - loss: 0.2557 - accuracy: 0.8947 - val_loss: 0.6630 - val_accuracy: 0.7473
Epoch 62/100
12/12 [==============================] - 0s 3ms/step - loss: 0.2548 - accuracy: 0.9003 - val_loss: 0.6509 - val_accuracy: 0.7692
Epoch 63/100
12/12 [==============================] - 0s 2ms/step - loss: 0.2511 - accuracy: 0.9058 - val_loss: 0.6466 - val_accuracy: 0.7802
Epoch 64/100
12/12 [==============================] - 0s 2ms/step - loss: 0.2496 - accuracy: 0.9114 - val_loss: 0.6570 - val_accuracy: 0.7692
Epoch 65/100
12/12 [==============================] - 0s 3ms/step - loss: 0.2481 - accuracy: 0.9169 - val_loss: 0.6770 - val_accuracy: 0.7692
Epoch 

In [194]:
test_loss, test_acc = model.evaluate(X_test, y_test)

print(f'Test Accuracy: {test_acc}')

4/4 [==============================] - 0s 1ms/step - loss: 1.1332 - accuracy: 0.7281
Test Accuracy: 0.7280701994895935
